Following [this](https://docs.pydantic.dev/usage/models/)

<br/>

**Pydantic Note**

*pydantic* is primarily a **parsing** library, **not a validation** library. Validation is a means to an end: building a model which conforms to the types and constraints provided.

> In other words, *pydantic* guarantees the *types and constraints of the output model, not the input data*.

This might sound like an esoteric distinction, but it is not. If you're unsure what this means or how it might affect your usage you should read the section about [Data Conversion](https://docs.pydantic.dev/usage/models/#data-conversion).

Although validation is not the main purpose of *pydantic*, you can use this library for custom [validation](https://docs.pydantic.dev/usage/validators/).

### Basic model usage

In [1]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name = "Jane Doe"

In [2]:
user = User(id="123")  # pyright: ignore

user

User(id=123, name='Jane Doe')

⬆️ Note the `# pyright: ignore` above, this is because PyRight/Lance doesn't understand *pydantic*'s automatic conversions.

Most of the time strict PyRight type checking is still useful, so a workaround is in order.

This is discussed in detail here:
https://docs.pydantic.dev/visual_studio_code/#strict-errors

> In general useful for VSCode and *pydantic* compatibility: https://docs.pydantic.dev/visual_studio_code/

Either use `# pyright: ignore` or `my_id: Any = "123"` then pass to function.

In [3]:
user_x = User(id="123.45")

user_x

# NOTE: This doesn't seem to agree with the docs. "123.45" does NOT get cast to 123.

ValidationError: 1 validation error for User
id
  value is not a valid integer (type=type_error.integer)

In [5]:
assert user.id == 123
assert isinstance(user.id, int)

In [6]:
assert user.name == "Jane Doe"

In [8]:
user.__fields_set__  # The fields which were supplied when user was initialised.

{'id'}

In [9]:
user.dict()  # "name" was automatically taken from the default value.

{'id': 123, 'name': 'Jane Doe'}

In [10]:
# Mutable.
user.id = 321
assert user.id == 321

### Recursive Models

In [12]:
from typing import List, Optional
from pydantic import BaseModel


class Foo(BaseModel):
    count: int
    size: Optional[float] = None


class Bar(BaseModel):
    apple = "x"
    banana = "y"


class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]


# NOTE: Initialised through dictionaries below. This is a pydantic thing.
m = Spam(foo={"count": 4}, bars=[{"apple": "x1"}, {"apple": "x2"}])  # pyright: ignore
# ^ As noted above re. PyRight.

print(m)
# > foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'),
# > Bar(apple='x2', banana='y')]
print(m.dict())
"""
{
    'foo': {'count': 4, 'size': None},
    'bars': [
        {'apple': 'x1', 'banana': 'y'},
        {'apple': 'x2', 'banana': 'y'},
    ],
}
""";

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]
{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


In [14]:
m = Spam(foo=Foo(count=4), bars=[Bar(apple="x1"), Bar(apple="x2")])  # pyright: ignore
# NOTE: Above, PyRight isn't happy about the `apple` initializer parameter in `Bar`.
# I guess this is dynamically created by BaseModel, but PyRight doesn't know it...

m

Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])

### ORM Mode (aka Arbitrary Class Instances)

Pydantic models can be created from arbitrary class instances to support models that map to ORM objects.

To do this:
* The Config property `orm_mode` must be set to `True`.
* The special constructor `from_orm` must be used to create the model instance.

The example here uses SQLAlchemy, but the same approach should work for any ORM.

In [15]:
from typing import List
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

Base = declarative_base()


class CompanyOrm(Base):  # SQLAlchemy ORM.
    __tablename__ = "companies"
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))


class CompanyModel(BaseModel):  # Pydantic model.
    id: int
    public_key: constr(max_length=20)  # pyright: ignore
    name: constr(max_length=63)  # pyright: ignore
    domains: List[constr(max_length=255)]  # pyright: ignore

    class Config:
        orm_mode = True  # NOTE.


co_orm = CompanyOrm(
    id=123,
    public_key="foobar",
    name="Testing",
    domains=["example.com", "foobar.com"],
)

print(co_orm)
# > <models_orm_mode.CompanyOrm object at 0x102154f40>
co_model = CompanyModel.from_orm(co_orm)  # NOTE this conversion.
print(co_model)
# > id=123 public_key='foobar' name='Testing' domains=['example.com',
# > 'foobar.com']

id=123 public_key='foobar' name='Testing' domains=['example.com', 'foobar.com']


* Next two subsections are too ORM specific, skipped.

### Error Handling

pydantic will raise `ValidationError` whenever it finds an error in the data it's validating.

> Validation code should not raise `ValidationError` itself, but rather raise 
`ValueError`, `TypeError` or `AssertionError` (or subclasses of `ValueError` or `TypeError`)
which will be caught and used to populate `ValidationError`.

One exception will be raised regardless of the number of errors found, that `ValidationError` will contain information about all the errors and how they happened.

In [17]:
from typing import List
from pydantic import BaseModel, ValidationError, conint


class Location(BaseModel):
    lat = 0.1
    lng = 10.1


class Model(BaseModel):
    is_required: float
    gt_int: conint(gt=42)  # pyright: ignore
    list_of_ints: List[int] = None  # pyright: ignore
    a_float: float = None  # pyright: ignore
    recursive_model: Location = None  # pyright: ignore


# NOTE: ^ It seems pydantic is happy with = None without specifying Optional[].


data = dict(
    list_of_ints=["1", 2, "bad"],
    a_float="not a float",
    recursive_model={"lat": 4.2, "lng": "New York"},
    gt_int=21,
)

try:
    Model(**data)  # pyright: ignore
except ValidationError as e:
    print(e)
    """
    5 validation errors for Model
    is_required
      field required (type=value_error.missing)
    gt_int
      ensure this value is greater than 42 (type=value_error.number.not_gt;
    limit_value=42)
    list_of_ints -> 2
      value is not a valid integer (type=type_error.integer)
    a_float
      value is not a valid float (type=type_error.float)
    recursive_model -> lng
      value is not a valid float (type=type_error.float)
    """

print("---")

try:
    Model(**data)  # pyright: ignore
except ValidationError as e:
    print(e.json())
    """
    [
      {
        "loc": [
          "is_required"
        ],
        "msg": "field required",
        "type": "value_error.missing"
      },
      {
        "loc": [
          "gt_int"
        ],
        "msg": "ensure this value is greater than 42",
        "type": "value_error.number.not_gt",
        "ctx": {
          "limit_value": 42
        }
      },
      {
        "loc": [
          "list_of_ints",
          2
        ],
        "msg": "value is not a valid integer",
        "type": "type_error.integer"
      },
      {
        "loc": [
          "a_float"
        ],
        "msg": "value is not a valid float",
        "type": "type_error.float"
      },
      {
        "loc": [
          "recursive_model",
          "lng"
        ],
        "msg": "value is not a valid float",
        "type": "type_error.float"
      }
    ]
    """

5 validation errors for Model
is_required
  field required (type=value_error.missing)
gt_int
  ensure this value is greater than 42 (type=value_error.number.not_gt; limit_value=42)
list_of_ints -> 2
  value is not a valid integer (type=type_error.integer)
a_float
  value is not a valid float (type=type_error.float)
recursive_model -> lng
  value is not a valid float (type=type_error.float)
---
[
  {
    "loc": [
      "is_required"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "gt_int"
    ],
    "msg": "ensure this value is greater than 42",
    "type": "value_error.number.not_gt",
    "ctx": {
      "limit_value": 42
    }
  },
  {
    "loc": [
      "list_of_ints",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  },
  {
    "loc": [
      "a_float"
    ],
    "msg": "value is not a valid float",
    "type": "type_error.float"
  },
  {
    "loc": [
      "recursive_model",
      "lng"
    

In your custom data types or validators you should use `ValueError`, `TypeError` or `AssertionError` to raise errors.

In [18]:
from pydantic import BaseModel, ValidationError, validator


class Model(BaseModel):
    foo: str

    @validator("foo")  # NOTE: Using @validator decorator.
    def value_must_equal_bar(cls, v):
        if v != "bar":
            raise ValueError('value must be "bar"')

        return v


try:
    Model(foo="ber")
except ValidationError as e:
    print(e.errors())
    """
    [
        {
            'loc': ('foo',),
            'msg': 'value must be "bar"',
            'type': 'value_error',
        },
    ]
    """

[{'loc': ('foo',), 'msg': 'value must be "bar"', 'type': 'value_error'}]


You can also define your **own error classes**, which can specify a custom error *code, message template, and context*:

In [19]:
from pydantic import BaseModel, PydanticValueError, ValidationError, validator


class NotABarError(PydanticValueError):  # NOTE: Custom validation error.
    code = "not_a_bar"
    msg_template = 'value is not "bar", got "{wrong_value}"'


class Model(BaseModel):
    foo: str

    @validator("foo")
    def value_must_equal_bar(cls, v):
        if v != "bar":
            raise NotABarError(wrong_value=v)  # NOTE: Raised here.
        return v


try:
    Model(foo="ber")
except ValidationError as e:
    print(e.json())
    """
    [
      {
        "loc": [
          "foo"
        ],
        "msg": "value is not \"bar\", got \"ber\"",
        "type": "value_error.not_a_bar",
        "ctx": {
          "wrong_value": "ber"
        }
      }
    ]
    """

[
  {
    "loc": [
      "foo"
    ],
    "msg": "value is not \"bar\", got \"ber\"",
    "type": "value_error.not_a_bar",
    "ctx": {
      "wrong_value": "ber"
    }
  }
]


### Helper Functions

Pydantic provides three classmethod helper functions on models for parsing data:

* `parse_obj`: this is very similar to the `__init__` method of the model, except it takes a dict rather than keyword arguments. If the object passed is not a dict a `ValidationError` will be raised.
* `parse_raw`: this takes a `str` or `bytes` and parses it as *json*, then passes the result to `parse_obj`. Parsing *pickle* data is also supported by setting the `content_type` argument appropriately.
* `parse_file`: this takes in a file path, reads the file and passes the contents to `parse_raw`. If `content_type` is omitted, it is inferred from the file's extension.


In [20]:
import pickle
from datetime import datetime
from pathlib import Path

from pydantic import BaseModel, ValidationError


class User(BaseModel):
    id: int
    name = "John Doe"
    signup_ts: datetime = None  # pyright: ignore


m = User.parse_obj({"id": 123, "name": "James"})  # NOTE.
print(m)
# > id=123 signup_ts=None name='James'

try:
    User.parse_obj(["not", "a", "dict"])  # NOTE.
except ValidationError as e:
    print(e)
    """
    1 validation error for User
    __root__
      User expected dict not list (type=type_error)
    """

# assumes json as no content type passed
m = User.parse_raw('{"id": 123, "name": "James"}')  # NOTE.
print(m)
# > id=123 signup_ts=None name='James'

pickle_data = pickle.dumps(
    {"id": 123, "name": "James", "signup_ts": datetime(2017, 7, 14)}
)
m = User.parse_raw(  # NOTE.
    pickle_data, content_type="application/pickle", allow_pickle=True
)
print(m)
# > id=123 signup_ts=datetime.datetime(2017, 7, 14, 0, 0) name='James'

path = Path("data.json")
path.write_text('{"id": 123, "name": "James"}')
m = User.parse_file(path)  # NOTE.
print(m)
# > id=123 signup_ts=None name='James'

id=123 signup_ts=None name='James'
1 validation error for User
__root__
  User expected dict not list (type=type_error)
id=123 signup_ts=None name='James'
id=123 signup_ts=datetime.datetime(2017, 7, 14, 0, 0) name='James'
id=123 signup_ts=None name='James'


#### Creating models without validation

`construct()` bypasses validation for speed (x30 faster). Only use when sure that data is already validated.

In [21]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    age: int
    name: str = "John Doe"


original_user = User(id=123, age=32)

user_data = original_user.dict()
print(user_data)
# > {'id': 123, 'age': 32, 'name': 'John Doe'}
fields_set = original_user.__fields_set__
print(fields_set)
# > {'id', 'age'}

# ...
# pass user_data and fields_set to RPC or save to the database etc.
# ...

# you can then create a new instance of User without
# re-running validation which would be unnecessary at this point:
new_user = User.construct(_fields_set=fields_set, **user_data)  # NOTE.
print(repr(new_user))
# > User(id=123, age=32, name='John Doe')
print(new_user.__fields_set__)
# > {'id', 'age'}

# construct can be dangerous, only use it with validated data!:
bad_user = User.construct(id="dog")  # NOTE.
print(repr(bad_user))
# > User(id='dog', name='John Doe')

{'id': 123, 'age': 32, 'name': 'John Doe'}
{'age', 'id'}
User(id=123, age=32, name='John Doe')
{'age', 'id'}
User(id='dog', name='John Doe')


### Generic Models

In order to declare a generic model, you perform the following steps:

* Declare one or more `typing.TypeVar` instances to use to parameterize your model.
* Declare a pydantic model that inherits from `pydantic.generics.GenericModel` and `typing.Generic`, where you pass the `TypeVar` instances as parameters to `typing.Generic`.
* Use the `TypeVar` instances as annotations where you will want to replace them with other types or pydantic models.


In [22]:
from typing import Generic, TypeVar, Optional, List

from pydantic import BaseModel, validator, ValidationError
from pydantic.generics import GenericModel

DataT = TypeVar("DataT")  # NOTE.


class Error(BaseModel):
    code: int
    message: str


class DataModel(BaseModel):
    numbers: List[int]
    people: List[str]


class Response(GenericModel, Generic[DataT]):  # NOTE.
    data: Optional[DataT]
    error: Optional[Error]

    @validator("error", always=True)
    def check_consistency(cls, v, values):
        if v is not None and values["data"] is not None:
            raise ValueError("must not provide both data and error")
        if v is None and values.get("data") is None:
            raise ValueError("must provide data or error")
        return v


data = DataModel(numbers=[1, 2, 3], people=[])
error = Error(code=404, message="Not found")

print(Response[int](data=1))  # pyright: ignore
# > data=1 error=None
print(Response[str](data="value"))  # pyright: ignore
# > data='value' error=None
print(Response[str](data="value").dict())  # pyright: ignore
# > {'data': 'value', 'error': None}
print(Response[DataModel](data=data).dict())  # pyright: ignore
"""
{
    'data': {'numbers': [1, 2, 3], 'people': []},
    'error': None,
}
"""

print(Response[DataModel](error=error).dict())  # pyright: ignore
"""
{
    'data': None,
    'error': {'code': 404, 'message': 'Not found'},
}
"""
try:
    Response[int](data="value")  # pyright: ignore
except ValidationError as e:
    print(e)
    """
    2 validation errors for Response[int]
    data
      value is not a valid integer (type=type_error.integer)
    error
      must provide data or error (type=value_error)
    """

data=1 error=None
data='value' error=None
{'data': 'value', 'error': None}
{'data': {'numbers': [1, 2, 3], 'people': []}, 'error': None}
{'data': None, 'error': {'code': 404, 'message': 'Not found'}}
2 validation errors for Response[int]
data
  value is not a valid integer (type=type_error.integer)
error
  must provide data or error (type=value_error)


**Note**

To inherit from a `GenericModel` without replacing the `TypeVar` instance, a class must also inherit from `typing.Generic`:

In [23]:
from typing import TypeVar, Generic
from pydantic.generics import GenericModel

TypeX = TypeVar("TypeX")


class BaseClass(GenericModel, Generic[TypeX]):
    X: TypeX


# ∧
# |
# |
class ChildClass(BaseClass[TypeX], Generic[TypeX]):  # NOTE.
    # Inherit from Generic[TypeX]
    pass


# Replace TypeX by int
print(ChildClass[int](X=1))
# > X=1

X=1


You can also create a generic subclass of a `GenericModel` that partially or fully replaces the type parameters in the superclass.

In [24]:
from typing import TypeVar, Generic
from pydantic.generics import GenericModel

TypeX = TypeVar("TypeX")
TypeY = TypeVar("TypeY")
TypeZ = TypeVar("TypeZ")


class BaseClass(GenericModel, Generic[TypeX, TypeY]):
    x: TypeX
    y: TypeY


# ∧
# |
# |
class ChildClass(
    BaseClass[int, TypeY], Generic[TypeY, TypeZ]
):  # NOTE: Look at this line carefully!
    z: TypeZ


# Replace TypeY by str
print(ChildClass[str, int](x=1, y="y", z=3))
# > x=1 y='y' z=3

x=1 y='y' z=3


If the name of the concrete subclasses is important, you can also override the default behavior:

In [25]:
from typing import Generic, TypeVar, Type, Any, Tuple

from pydantic.generics import GenericModel

DataT = TypeVar("DataT")


class Response(GenericModel, Generic[DataT]):
    data: DataT

    @classmethod
    def __concrete_name__(
        cls: Type[Any], params: Tuple[Type[Any], ...]
    ) -> str:  # NOTE.
        return f"{params[0].__name__.title()}Response"


print(repr(Response[int](data=1)))
# > IntResponse(data=1)
print(repr(Response[str](data="a")))
# > StrResponse(data='a')

IntResponse(data=1)
StrResponse(data='a')


Using the same `TypeVar` in nested models allows you to enforce typing relationships at different points in your model:

In [26]:
from typing import Generic, TypeVar

from pydantic import ValidationError
from pydantic.generics import GenericModel

T = TypeVar("T")


class InnerT(GenericModel, Generic[T]):
    inner: T


class OuterT(GenericModel, Generic[T]):
    outer: T
    nested: InnerT[T]


nested = InnerT[int](inner=1)
print(OuterT[int](outer=1, nested=nested))
# > outer=1 nested=InnerT[int](inner=1)
try:
    nested = InnerT[str](inner="a")
    print(OuterT[int](outer="a", nested=nested))  # NOTE here.
except ValidationError as e:
    print(e)
    """
    2 validation errors for OuterT[int]
    outer
      value is not a valid integer (type=type_error.integer)
    nested -> inner
      value is not a valid integer (type=type_error.integer)
    """

outer=1 nested=InnerT[int](inner=1)
2 validation errors for OuterT[int]
outer
  value is not a valid integer (type=type_error.integer)
nested -> inner
  value is not a valid integer (type=type_error.integer)


In [27]:
# Bounds are supported.

from typing import Generic, TypeVar

from pydantic import ValidationError
from pydantic.generics import GenericModel

AT = TypeVar("AT")
BT = TypeVar("BT")


class Model(GenericModel, Generic[AT, BT]):
    a: AT
    b: BT


print(Model(a="a", b="a"))
# > a='a' b='a'

IntT = TypeVar("IntT", bound=int)
typevar_model = Model[int, IntT]
print(typevar_model(a=1, b=1))  # pyright: ignore
# > a=1 b=1
try:
    typevar_model(a="a", b="a")
except ValidationError as exc:
    print(exc)
    """
    2 validation errors for Model[int, IntT]
    a
      value is not a valid integer (type=type_error.integer)
    b
      value is not a valid integer (type=type_error.integer)
    """

concrete_model = typevar_model[int]
print(concrete_model(a=1, b=1))
# > a=1 b=1

a='a' b='a'
a=1 b=1
2 validation errors for Model[int, IntT]
a
  value is not a valid integer (type=type_error.integer)
b
  value is not a valid integer (type=type_error.integer)
a=1 b=1


### Dynamic model creation

In [28]:
# Here StaticFoobarModel and DynamicFoobarModel are identical.

from pydantic import BaseModel, create_model

DynamicFoobarModel = create_model(
    "DynamicFoobarModel", foo=(str, ...), bar=123
)  # NOTE the `...` default value.
# ^ Fields are defined by either a tuple of the form (<type>, <default value>) or just a default value.


class StaticFoobarModel(BaseModel):
    foo: str
    bar: int = 123

In [30]:
# foo=(str, ...) above ensures that foo needs to be specified, hence below is an error.

d = DynamicFoobarModel()
d

ValidationError: 1 validation error for DynamicFoobarModel
foo
  field required (type=value_error.missing)

In [31]:
# The special key word arguments __config__ and __base__ can be used to customise the new model.
# This includes extending a base model with extra fields.

from pydantic import BaseModel, create_model


class FooModel(BaseModel):
    foo: str
    bar: int = 123


BarModel = create_model(
    "BarModel",
    apple="russet",
    banana="yellow",
    __base__=FooModel,
)
print(BarModel)
# > <class 'pydantic.main.BarModel'>
print(BarModel.__fields__.keys())
# > dict_keys(['foo', 'bar', 'apple', 'banana'])

<class 'pydantic.main.BarModel'>
dict_keys(['foo', 'bar', 'apple', 'banana'])


In [32]:
# You can also add validators by passing a dict to the __validators__ argument.

from pydantic import create_model, ValidationError, validator


def username_alphanumeric(cls, v):
    assert v.isalnum(), "must be alphanumeric"
    return v


validators = {"username_validator": validator("username")(username_alphanumeric)}

UserModel = create_model("UserModel", username=(str, ...), __validators__=validators)

user = UserModel(username="scolvin")
print(user)
# > username='scolvin'

try:
    UserModel(username="scolvi%n")
except ValidationError as e:
    print(e)
    """
    1 validation error for UserModel
    username
      must be alphanumeric (type=assertion_error)
    """

username='scolvin'
1 validation error for UserModel
username
  must be alphanumeric (type=assertion_error)


### Model creation from `NamedTuple` or `TypedDict`

Sometimes you already use in your application classes that inherit from `NamedTuple` or `TypedDict`
and you don't want to duplicate all your information to have a `BaseModel`.

For this pydantic provides `create_model_from_namedtuple` and `create_model_from_typeddict` methods.
Those methods have the exact same keyword arguments as `create_model`.

In [33]:
from typing_extensions import TypedDict

from pydantic import ValidationError, create_model_from_typeddict


class User(TypedDict):
    name: str
    id: int


class Config:
    extra = "forbid"


UserM = create_model_from_typeddict(User, __config__=Config)  # NOTE.
print(repr(UserM(name=123, id="3")))
# > User(name='123', id=3)

try:
    UserM(name=123, id="3", other="no")
except ValidationError as e:
    print(e)
    """
    1 validation error for User
    other
      extra fields not permitted (type=value_error.extra)
    """

User(name='123', id=3)
1 validation error for User
other
  extra fields not permitted (type=value_error.extra)


### Custom Root Types

In [2]:
from typing import List
import json
from pydantic import BaseModel
from pydantic.schema import schema


class Pets(BaseModel):
    __root__: List[str]  # NOTE.


print(Pets(__root__=["dog", "cat"]))
# > __root__=['dog', 'cat']
print(Pets(__root__=["dog", "cat"]).json())
# > ["dog", "cat"]
print(Pets.parse_obj(["dog", "cat"]))
# > __root__=['dog', 'cat']
print(Pets.schema())
"""
{
    'title': 'Pets',
    'type': 'array',
    'items': {'type': 'string'},
}
"""

pets_schema = schema([Pets])
print(json.dumps(pets_schema, indent=2))
"""
{
  "definitions": {
    "Pets": {
      "title": "Pets",
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  }
}
""";

__root__=['dog', 'cat']
["dog", "cat"]
__root__=['dog', 'cat']
{'title': 'Pets', 'type': 'array', 'items': {'type': 'string'}}
{
  "definitions": {
    "Pets": {
      "title": "Pets",
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  }
}


* The rest of this section is too specific, skipped.

### Faux Immutability

Models can be configured to be immutable via `allow_mutation = False`.

> Immutability in Python is never strict. If developers are determined/stupid they can always modify a so-called "immutable" object.

In [3]:
from pydantic import BaseModel


class FooBarModel(BaseModel):
    a: str
    b: dict

    class Config:
        allow_mutation = False  # NOTE.


foobar = FooBarModel(a="hello", b={"apple": "pear"})

try:
    foobar.a = "different"
except TypeError as e:
    print(e)
    # > "FooBarModel" is immutable and does not support item assignment

print(foobar.a)
# > hello
print(foobar.b)
# > {'apple': 'pear'}
foobar.b["apple"] = "grape"
print(foobar.b)
# > {'apple': 'grape'}

"FooBarModel" is immutable and does not support item assignment
hello
{'apple': 'pear'}
{'apple': 'grape'}


### Abstract Base Classes

👍 Pydantic models can be used alongside Python's Abstract Base Classes (ABCs).

In [4]:
import abc
from pydantic import BaseModel


class FooBarModel(BaseModel, abc.ABC):  # NOTE.
    a: str
    b: int

    @abc.abstractmethod
    def my_abstract_method(self):
        pass

### Field Ordering

Field order is important in models for the following reasons:
* validation is performed in the order fields are defined; fields validators can access the values of earlier fields, but not later ones
* field order is preserved in the model schema
* field order is preserved in validation errors
* field order is preserved by `.dict()` and `.json()` etc.

In [5]:
from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    a: int
    b = 2  # NOTE: The non-annotated fields combined with annotated field(s) is NOT recommended.
    c: int = 1
    d = 0
    e: float


print(Model.__fields__.keys())
# > dict_keys(['a', 'c', 'e', 'b', 'd'])
m = Model(e=2, a=1)
print(m.dict())
# > {'a': 1, 'c': 1, 'e': 2.0, 'b': 2, 'd': 0}
try:
    Model(a="x", b="x", c="x", d="x", e="x")  # pyright: ignore
except ValidationError as e:
    error_locations = [e["loc"] for e in e.errors()]

print(error_locations)  # pyright: ignore
# > [('a',), ('c',), ('e',), ('b',), ('d',)]

dict_keys(['a', 'c', 'e', 'b', 'd'])
{'a': 1, 'c': 1, 'e': 2.0, 'b': 2, 'd': 0}
[('a',), ('c',), ('e',), ('b',), ('d',)]


> As demonstrated by the example above, combining the use of annotated and non-annotated fields in the same model 
> can result in surprising field orderings. (This is due to limitations of Python)
> 
> Therefore, we recommend adding type annotations to all fields, even when a default value would determine
> the type by itself to guarantee field order is preserved.

### Required Fields

In [6]:
from pydantic import BaseModel, Field


class Model(BaseModel):
    a: int
    b: int = ...  # NOTE: Will not work with mypy and is NOT recommended.
    c: int = Field(...)

#### Required Optional fields

If you want to specify a field that can take a `None` value **while still being required**, you can use `Optional` with `...`:

In [10]:
from typing import Optional
from pydantic import BaseModel, Field, ValidationError


class Model(BaseModel):
    a: Optional[int]
    b: Optional[int] = ...  # NOTE: Optional with `...`. Not recommended!
    c: Optional[int] = Field(...)  # NOTE: Optional with `...`.


print(Model(b=1, c=2))
# > a=None b=1 c=2
try:
    Model(a=1, b=2)
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    c
      field required (type=value_error.missing)
    """

a=None b=1 c=2
1 validation error for Model
c
  field required (type=value_error.missing)


In this model, `a`, `b`, and `c` can take `None` as a value.

But `a` is optional, while `b` and `c` are required. `b` and `c` require a value, even if the value is `None`.

### Field with dynamic default value *[Beta]*

When declaring a field with a default value, you may want it to be dynamic (i.e. different for each model).

To do this, you may want to use a `default_factory`.

In [11]:
from datetime import datetime
from uuid import UUID, uuid4
from pydantic import BaseModel, Field


class Model(BaseModel):
    uid: UUID = Field(
        default_factory=uuid4
    )  # NOTE: Dynamic default value (generated every instance!).
    updated: datetime = Field(
        default_factory=datetime.utcnow
    )  # NOTE: Dynamic default value (generated every instance!).


m1 = Model()
m2 = Model()
print(f"{m1.uid} != {m2.uid}")
# > aac7f12d-a774-445d-ac39-aba7ef127532 != f529cbec-ca6d-4313-a796-e2948c1ab6b7
print(f"{m1.updated} != {m2.updated}")
# > 2022-12-06 14:32:40.915017 != 2022-12-06 14:32:40.915025

17ec95e2-90b5-4906-93ec-5769d4791610 != 202ca745-e7e0-4e7e-ab9c-ba53cffa6c5e
2022-12-18 15:53:32.721526 != 2022-12-18 15:53:32.721568


### Automatically excluded attributes

> Class variables which
>    * begin with an underscore 
>    * and attributes annotated with `typing.ClassVar` 
> 
> will be automatically *excluded* from the model!


### Private model attributes

> ❓ I don't understand how this is different from the previous section. Are these private but not excluded?


If you need to vary or manipulate internal attributes on instances of the model,
you can declare them using `PrivateAttr`:

In [12]:
from datetime import datetime
from random import randint

from pydantic import BaseModel, PrivateAttr


class TimeAwareModel(BaseModel):
    _processed_at: datetime = PrivateAttr(default_factory=datetime.now)  # NOTE.
    _secret_value: str = PrivateAttr()  # NOTE.

    def __init__(self, **data):
        super().__init__(**data)
        # this could also be done with default_factory
        self._secret_value = randint(1, 5)  # pyright: ignore


m = TimeAwareModel()
print(m._processed_at)
# > 2022-12-06 14:32:41.180747
print(m._secret_value)
# > 3

2022-12-18 15:59:20.830937
2


Private attribute names must start with underscore to prevent conflicts with model fields:
both `_attr` and `__attr__` are supported.

If `Config.underscore_attrs_are_private` is `True`, any non-ClassVar underscore attribute will be treated as private:

> Again I don't understand the difference here: "class variable" vs "private attribute"?

In [13]:
from typing import ClassVar

from pydantic import BaseModel


class Model(BaseModel):
    _class_var: ClassVar[str] = "class var value"
    _private_attr: str = "private attr value"

    class Config:
        underscore_attrs_are_private = True


print(Model._class_var)
# > class var value
print(Model._private_attr)
# > <member '_private_attr' of 'Model' objects>
print(Model()._private_attr)
# > private attr value

class var value
<member '_private_attr' of 'Model' objects>
private attr value


> Upon class creation pydantic constructs `__slots__` filled with private attributes.

### Parsing data into a specified type

Pydantic includes a standalone utility function `parse_obj_as` that can be used to apply the parsing logic used to
populate pydantic models in a more ad-hoc way.

This function behaves similarly to `BaseModel.parse_obj`, but works with arbitrary pydantic-compatible types.

This is especially useful when you want to parse results into **a type that is not a direct subclass of `BaseModel`**.

For example: 

In [14]:
from typing import List

from pydantic import BaseModel, parse_obj_as


class Item(BaseModel):
    id: int
    name: str


# `item_data` could come from an API call, eg., via something like:
# item_data = requests.get('https://my-api.com/items').json()
item_data = [{"id": 1, "name": "My Item"}]

items = parse_obj_as(
    List[Item], item_data
)  # NOTE: Parsing into List[Item], NOT by itself a subclass of BaseModel.
print(items)
# > [Item(id=1, name='My Item')]

[Item(id=1, name='My Item')]


### Data Conversion

> ⚠️ *pydantic* may cast input data to force it to conform to model field types,
> and in some cases *this may result in a loss of information*.

For example:

In [1]:
from pydantic import BaseModel


class Model(BaseModel):
    a: int
    b: float
    c: str


print(Model(a=3.1415, b=" 2.72 ", c=123).dict())  # pyright: ignore
# > {'a': 3, 'b': 2.72, 'c': '123'}

{'a': 3, 'b': 2.72, 'c': '123'}


### Model signature

In [2]:
import inspect
from pydantic import BaseModel, Field


class FooModel(BaseModel):
    id: int
    name: str = None  # pyright: ignore
    description: str = "Foo"
    apple: int = Field(..., alias="pear")


print(inspect.signature(FooModel))  # NOTE.
# > (*, id: int, name: str = None, description: str = 'Foo', pear: int) -> None

(*, id: int, name: str = None, description: str = 'Foo', pear: int) -> None


In [3]:
# The generated signature will also respect custom __init__ functions:

import inspect

from pydantic import BaseModel


class MyModel(BaseModel):
    id: int
    info: str = "Foo"

    def __init__(self, id: int = 1, *, bar: str, **data) -> None:  # NOTE.
        """My custom init!"""
        super().__init__(id=id, bar=bar, **data)


print(inspect.signature(MyModel))  # NOTE.
# > (id: int = 1, *, bar: str, info: str = 'Foo') -> None

(id: int = 1, *, bar: str, info: str = 'Foo') -> None


### Structural pattern matching
* Python 3.10 thing.